In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta
from IPython.display import HTML, display

class LibraryManagement:
    def __init__(self):
        self.books_file = 'books.csv'
        self.borrowed_file = 'borrowed_books.csv'
        if os.path.exists(self.books_file):
            self.books = pd.read_csv(self.books_file)
        else:
            self.books = pd.DataFrame(columns=['ID', 'Title', 'Author', 'Status'])
        if os.path.exists(self.borrowed_file):
            self.borrowed = pd.read_csv(self.borrowed_file)
        else:
            self.borrowed = pd.DataFrame(columns=['BookID', 'Borrower', 'BorrowDate', 'DueDate'])
        self.message = ""
        self.search_results = None

    def save_data(self):
        self.books.to_csv(self.books_file, index=False)
        self.borrowed.to_csv(self.borrowed_file, index=False)

    def add_book(self, title, author):
        if not title or not author:
            self.message = "Title and author cannot be empty."
            return
        new_id = len(self.books) + 1
        new_book = pd.DataFrame({
            'ID': [new_id],
            'Title': [title],
            'Author': [author],
            'Status': ['Available']
        })
        self.books = pd.concat([self.books, new_book], ignore_index=True)
        self.save_data()
        self.message = f"Book '{title}' added successfully with ID {new_id}"

    def view_books(self):
        if self.books.empty:
            self.message = "No books in the library."
            return "<p>No books in the library.</p>"
        else:
            self.message = ""
            return self.books.to_html(index=False, classes="table table-striped")

    def borrow_book(self, book_id, borrower):
        try:
            book_id = int(book_id)
            if not borrower:
                self.message = "Borrower name cannot be empty."
                return
            if book_id not in self.books['ID'].values:
                self.message = "Book ID not found."
                return
            book_status = self.books.loc[self.books['ID'] == book_id, 'Status'].iloc[0]
            if book_status == 'Borrowed':
                self.message = "Book is already borrowed."
                return
            self.books.loc[self.books['ID'] == book_id, 'Status'] = 'Borrowed'
            borrow_date = datetime.now().strftime('%Y-%m-%d')
            due_date = (datetime.now() + timedelta(days=14)).strftime('%Y-%m-%d')
            new_borrow = pd.DataFrame({
                'BookID': [book_id],
                'Borrower': [borrower],
                'BorrowDate': [borrow_date],
                'DueDate': [due_date]
            })
            self.borrowed = pd.concat([self.borrowed, new_borrow], ignore_index=True)
            self.save_data()
            self.message = f"Book ID {book_id} borrowed by {borrower}. Due date: {due_date}"
        except ValueError:
            self.message = "Book ID must be a number."

    def return_book(self, book_id):
        try:
            book_id = int(book_id)
            if book_id not in self.books['ID'].values:
                self.message = "Book ID not found."
                return
            if book_id not in self.borrowed['BookID'].values:
                self.message = "Book is not borrowed."
                return
            self.books.loc[self.books['ID'] == book_id, 'Status'] = 'Available'
            self.borrowed = self.borrowed[self.borrowed['BookID'] != book_id]
            self.save_data()
            self.message = f"Book ID {book_id} returned successfully."
        except ValueError:
            self.message = "Book ID must be a number."

    def search_book(self, keyword):
        if not keyword:
            self.message = "Keyword cannot be empty."
            self.search_results = None
            return
        results = self.books[self.books['Title'].str.contains(keyword, case=False, na=False) |
                             self.books['Author'].str.contains(keyword, case=False, na=False)]
        if results.empty:
            self.message = "No books found."
            self.search_results = None
        else:
            self.message = ""
            self.search_results = results.to_html(index=False, classes="table table-striped")

def generate_html(library):
    html_content = f"""
    <html>
    <head>
        <style>
            body {{
                font-family: Arial, sans-serif;
                max-width: 800px;
                margin: 20px auto;
                padding: 20px;
                background-color: #f4f4f4;
            }}
            h1 {{
                text-align: center;
                color: #333;
            }}
            .container {{
                background-color: white;
                padding: 20px;
                border-radius: 8px;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
            }}
            .form-group {{
                margin-bottom: 15px;
            }}
            label {{
                display: block;
                margin-bottom: 5px;
                font-weight: bold;
            }}
            input[type="text"], input[type="number"] {{
                width: 100%;
                padding: 8px;
                border: 1px solid #ccc;
                border-radius: 4px;
            }}
            button {{
                padding: 10px 20px;
                background-color: #007BFF;
                color: white;
                border: none;
                border-radius: 4px;
                cursor: pointer;
                margin-right: 10px;
            }}
            button:hover {{
                background-color: #0056b3;
            }}
            .message {{
                color: green;
                margin-bottom: 15px;
            }}
            .error {{
                color: red;
                margin-bottom: 15px;
            }}
            .table {{
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
            }}
            .table th, .table td {{
                border: 1px solid #ddd;
                padding: 8px;
                text-align: left;
            }}
            .table th {{
                background-color: #007BFF;
                color: white;
            }}
            .table-striped tr:nth-child(even) {{
                background-color: #f2f2f2;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Library Management System</h1>
            <div class="{ 'error' if library.message.startswith(('Book ID', 'No books', 'Keyword', 'Title', 'Borrower')) else 'message' }">
                {library.message}
            </div>
            <form id="libraryForm">
                <input type="hidden" id="action" name="action" value="">
                <div class="form-group">
                    <label for="title">Book Title:</label>
                    <input type="text" id="title" name="title">
                </div>
                <div class="form-group">
                    <label for="author">Author:</label>
                    <input type="text" id="author" name="author">
                </div>
                <div class="form-group">
                    <label for="book_id">Book ID:</label>
                    <input type="number" id="book_id" name="book_id">
                </div>
                <div class="form-group">
                    <label for="borrower">Borrower Name:</label>
                    <input type="text" id="borrower" name="borrower">
                </div>
                <div class="form-group">
                    <label for="keyword">Search Keyword:</label>
                    <input type="text" id="keyword" name="keyword">
                </div>
                <button type="submit" formaction="?action=add">Add Book</button>
                <button type="submit" formaction="?action=view">View Books</button>
                <button type="submit" formaction="?action=borrow">Borrow Book</button>
                <button type="submit" formaction="?action=return">Return Book</button>
                <button type="submit" formaction="?action=search">Search Book</button>
            </form>
            <div>
                {library.search_results if library.search_results else library.view_books()}
            </div>
        </div>
    </body>
    </html>
    """
    return html_content

def main():
    library = LibraryManagement()
    try:
        action = input("Enter action (add/view/borrow/return/search, or leave blank to display UI): ").strip().lower()
    except EOFError:
        action = ""
    if action:
        if action == 'add':
            try:
                title = input("Enter book title: ").strip()
                author = input("Enter book author: ").strip()
                library.add_book(title, author)
            except EOFError:
                library.message = "Input error for title or author."
        elif action == 'view':
            display(HTML(library.view_books()))
        elif action == 'borrow':
            try:
                book_id = input("Enter book ID: ").strip()
                borrower = input("Enter borrower name: ").strip()
                library.borrow_book(book_id, borrower)
            except EOFError:
                library.message = "Input error for book ID or borrower name."
        elif action == 'return':
            try:
                book_id = input("Enter book ID: ").strip()
                library.return_book(book_id)
            except EOFError:
                library.message = "Input error for book ID."
        elif action == 'search':
            try:
                keyword = input("Enter search keyword: ").strip()
                library.search_book(keyword)
            except EOFError:
                library.message = "Input error for search keyword."
        else:
            library.message = "Invalid action."
    display(HTML(generate_html(library)))

if __name__ == "__main__":
    main()

Enter action (add/view/borrow/return/search, or leave blank to display UI): add
Enter book title: applied mathematics
Enter book author: venky


ID,Title,Author,Status
1,applied mathematics,venky,Available
